In [1]:
# General imports
import os
import sys
import json
import joblib
import numpy as np

# MLServer folder path
thisdir = os.getcwd()
mlserverdir = os.path.join(thisdir, 'mlserver-model')
sys.path.append(mlserverdir)

# Import PixelRing2NMF and NMF2D
from pixelring2nmf import PixelRing2NMF
from nmf2d import NMF2D

## Load model

In [2]:
# Model path
nmf_file = '/eos/user/a/alaperto/SWAN_projects/NMFolder/models/model_7_PXRing_2_period_3_type_1.pkl'

# Load model & print shape
nmf_model = joblib.load(nmf_file)
print(f"Loaded NMF2D model: {nmf_file}")
print(f"Ring 2 Model X-Y Shape: {nmf_model.xshape}")

Loaded NMF2D model: /eos/user/a/alaperto/SWAN_projects/NMFolder/models/model_7_PXRing_2_period_3_type_1.pkl
Ring 2 Model X-Y Shape: [136, 48]


/cvmfs/sft.cern.ch/lcg/views/LCG_108_swan/x86_64-el9-gcc13-opt/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MiniBatchNMF from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Import data

In [3]:
import pickle

# Open pickle file (750 LS, 1 anomalous)
with open("/eos/user/a/alaperto/TkDQM/Devel/model_to_DIALS/dism-examples/src/sklearn_nmf/test_data.pkl", "rb") as f:
    dummy_input = pickle.load(f)

# DCS flags from OMS
oms_data = {
    'oms__run_number': np.array([386661]),
    'oms__lumisection_number': np.array([103]),
    'oms__fpix_ready': np.array([True]),
    'oms__beams_stable': np.array([True]),
    'oms__tob_ready': np.array([True]),
    'oms__pileup': np.array([63.55848067]),
    'oms__tecp_ready': np.array([True]),
    'oms__bpix_ready': np.array([True]),
    'oms__tecm_ready': np.array([True]),
    'oms__cms_active': np.array([True]),
    'oms__tibtid_ready': np.array([True])
}

#Multiply dummy OMS data for the number of LS
nLS = 750
oms_data_multi = {key: np.full(nLS, val.item()) if isinstance(val, np.ndarray) else np.full(nLS, val)
                  for key, val in oms_data.items()}

# Merge real data and OMS data
dummy_input.update(oms_data_multi)

## Execute (and save) the model

In [4]:
# Create model
pixel_model = PixelRing2NMF({'Ring2': nmf_model})

# Execute the prediction
flags = pixel_model.predict(dummy_input)
print("Output shape (all LS tested):", flags.shape)

# Save in joblib for MLServer
joblib.dump(pixel_model, os.path.join(mlserverdir, 'pixelring2nmf.joblib'))
print("PixelRing2NMF saved in joblib.")

[INFO]: Flagging results (before filtering): flagged 1 out of 750 lumisections.
[INFO]: Filtering: selected 750 out of 750 lumisections.
[INFO]: Flagging results (after filtering): flagged 1 out of 750 lumisections.
Output shape (all LS tested): (750,)
PixelRing2NMF saved in joblib.
